In [97]:
import pandas as pd
import numpy as np
from tqdm import trange

## Load and Organize Data

In [8]:
cannabis_df = pd.read_csv("cannabis_full.csv")
cannabis_df.head()

,Strain,Type,Rating,Effects,Flavor,Creative,Energetic,Tingly,Euphoric,Relaxed,...,Ammonia,Minty,Tree,Fruit,Butter,Pineapple,Tar,Rose,Plum,Pear
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange",0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
def split_dataset(data, target_column, test_ratio):
    np.random.seed(42)
    num_test = int(len(data) * test_ratio)
    
    shuffled_indices = np.random.permutation(len(data))
    test_indices = shuffled_indices[:num_test]
    train_indices = shuffled_indices[num_test:]

    train_df = data.iloc[train_indices]
    test_df = data.iloc[test_indices]

    X_train = train_df.drop(target_column, axis=1)
    y_train = train_df[target_column]

    X_test = test_df.drop(target_column, axis=1)
    y_test = test_df[target_column]

    return X_train, y_train, X_test, y_test

In [28]:
c_df = cannabis_df.dropna()
c_df = c_df[c_df["Type"].isin(["sativa", "indica"])]
c_df["Type_is_sativa"] = np.where(c_df["Type"] == "sativa", 1, 0)
c_df = c_df.drop(["Strain", "Effects", "Flavor", "Type"], axis=1)

X_train, y_train, X_test, y_test = split_dataset(c_df, target_column="Type_is_sativa", test_ratio=0.2)
X_train["intercept"] = 1
X_test["intercept"] = 1

In [33]:
len(X_train.columns)

66

## NN Functions

In [11]:
# Activation Functions

def relu(x):
    return np.maximum(0, x)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softplus(x):
    return np.log(1 + np.exp(x))

In [50]:
# Loss Functions

def squared_error(y, yhat):
    n = len(y)
    l = 1 / n * np.power(np.sum(yhat - y), 2)
    return l

In [53]:
# Gradient Functions

def squared_error_grad(y, yhat):
    n = len(y)
    dl = 2 / n * np.sum(yhat - y)
    return dl

def sigmoid_grad(z):
    left = np.power(1 / (1 + np.exp(-z)), 2)
    right = -np.exp(-z)
    return left * right

def softplus_grad(n):
    num = np.exp(n)
    den = np.exp(n) + 1
    return num / den

In [101]:
def forward_pass(X, W, V, activation_f):
    Z = X.dot(W)
    Q = activation_f(Z)
    n = Q.dot(V)
    p = activation_f(n)
    
    return Z, Q, n, p


def backpropogation(v, eta, activation_gradient_f, loss_gradient_f):
    dl = loss_gradient_f(v['y'], v['p'])
    dp = activation_gradient_f(v['n'])
    dn = v['Q']
    V_i = (dl * dp).dot(dn)

    dv = activation_gradient_f(v['Z'])
    dz = v['X']
    W_i = ((V_i * dv).T @ dz).T

    assert W.shape == W_i.shape
    assert V.shape == V_i.shape

    V += V_i * eta
    W += W_i * eta

    return W, V


# https://numpy.org/doc/stable/reference/generated/numpy.allclose.html
def stopping_condition(W, W_new, V, V_new):
    return np.allclose(W, W_new) and np.allclose(V, V_new)


def fit(X, y, eta, activation_funcs, loss_funcs, max_iter=1e6):
    W = np.random.random((X.shape[1], 3))
    V = np.random.random(3)
    for _ in trange(int(max_iter)):
        Z, Q, n, p = forward_pass(X, W, V, activation_funcs['activation_f'])

        values = {'W': W, 'V': V, 'Z': Z, 'Q': Q, 'n': n, 'p': p, 'X': X, 'y': y}

        W_new, V_new = backpropogation(values, eta, activation_funcs['gradient_f'], loss_funcs['gradient_f'])

        while not stopping_condition(W, W_new, V, V_new):
            return W_new, V_new
        
        W = W_new
        V = V_new

    return W, V

sigmoid_funcs = {'activation_f': sigmoid, 'gradient_f': sigmoid_grad}
squared_error_funcs = {'loss_f': squared_error, 'gradient_f': squared_error_grad}
fit(X_train.values, y_train.values, eta=0.01, activation_funcs=sigmoid_funcs, loss_funcs=squared_error_funcs)

  0%|          | 0/1000000 [00:00<?, ?it/s]


UnboundLocalError: cannot access local variable 'W' where it is not associated with a value

## Q1. NN with Squared-Error Loss